**Link to Training Notebook** [ https://www.kaggle.com/virajkadam/plant-pathology-apple-leaf-diseases ]

# Imports

In [1]:
import pandas as pd
import numpy as np
import os 
import gc 
from PIL import Image


from tensorflow.keras.models import load_model
from keras.preprocessing.image import load_img,img_to_array

from tensorflow_addons.metrics import F1Score

**test Directory**

In [2]:
test_dir='../input/plant-pathology-2021-fgvc8/test_images'

img_size=(256,256)

In [3]:
sample_sub=pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
sample_sub

,image,labels
0,85f8cb619c66b863.jpg,healthy
1,ad8770db05586b59.jpg,healthy
2,c7b03e718489f3ca.jpg,healthy


**metric**

In [4]:
f1_score=F1Score(num_classes=6,average='macro',name='f1_score')

# **Load Image**

In [5]:
def load_images(test_path,image):
    '''load image from given path'''
    img=load_img(test_path +'/' + image)
    img=img.resize(img_size)
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=img/255
    
    return img

In [6]:
label_classes=['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']

# Loading pre-trained models (on folds)

In [7]:
# loading pretrained models
model1=load_model('../input/plant-pathology-apple-leaf-diseases/model_0.h5')
model2=load_model('../input/plant-pathology-apple-leaf-diseases/model_1.h5')
model3=load_model('../input/plant-pathology-apple-leaf-diseases/model_2.h5')

In [8]:
def get_label(prediction_prob,thresh=0.3):
    '''get label for a class that satisfies given threshold'''
    
    prediction_prob=prediction_prob[0]
    prediction_prob=list(prediction_prob)
    labels=[label_classes[x] for x,prob in enumerate(prediction_prob) if prob >=thresh]
    
    labels=' '.join(labels)
    
    return labels

In [9]:
def predict(test_path,threshold):
    '''predict on test set, using given threshold'''
    
    images,labels=[],[]
    
    for image in os.listdir(test_path):
        
        #loading images         
        img=load_images(test_path,image)
        
        #mean prediction from 3 models
        pred_prob=(1/3* model1.predict(img)) + (1/3* model2.predict(img)) + (1/3* model3.predict(img))
        
        #getting prediction
        preds=get_label(pred_prob,thresh=threshold)
        
        #saving image name and predicted label
        images.append(image)
        labels.append(preds)
        
    return images,labels
        


In [10]:
#predicting using above func
image_ids,labels=predict(test_dir,threshold=0.2)

In [11]:
#submission file:
submission_file=pd.DataFrame({'image':image_ids,'labels':labels})

#saving submissions
submission_file.to_csv('submission.csv',index=False)

In [12]:
submission_file

,image,labels
0,ad8770db05586b59.jpg,complex frog_eye_leaf_spot rust
1,c7b03e718489f3ca.jpg,complex frog_eye_leaf_spot rust
2,85f8cb619c66b863.jpg,complex frog_eye_leaf_spot rust
